# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '⛰', '⛰', '⛰', '🗻', '🗻', '⛰', '⛰'],
['🌾', '🌾', '🌾', '🌾', '⛰', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '⛰', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '🗻', '🗻', '🗻', '⛰', '🌾'],
['🌾', '⛰', '⛰', '⛰', '🗻', '🗻', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🗻', '⛰', '🌾', '🌾'],
['🌾', '⛰', '⛰', '🗻', '🗻', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '⛰', '⛰', '🗻', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🗻', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '⛰', '🗻', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌲', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '⛰', '⛰', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🗻', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '⛰', '⛰', '🌾', '🐊', '🌾', '⛰', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🗻', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '⛰', '⛰', '🗻', '🗻', '🌾', '🗻', '🗻', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🗻', '🗻', '🗻', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🗻', '🗻', '🗻', '🗻', '⛰', '⛰', '⛰', '⛰', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾']
]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [2]:
test_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [3]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [4]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7, '🗻': 99999}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (it does not return anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [5]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="find_successors"></a>
## find_successors

This will use the valid moves to find all successors for a given starting point. There is logic to consider staying within world boundaries.

* **point** Tuple[int, int]: the current location of the bot, `(x, y)`.
* **world** List[List[str]]: the actual context for the navigation problem.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.

**returns** List[Tuple[int,int]]: the list of successor points to the current point

In [6]:
def find_successors(point: Tuple[int,int], world: List[List[str]], moves: List[Tuple[int,int]]):
    successors = []
    for move in moves:
        new_point = (point[0] + move[0], point[1] + move[1])
        if new_point[0] < 0 or new_point[1] < 0:
            continue
        elif new_point[0] >= len(world) or new_point[1] >= len(world[0]):
            continue
        successors.append(new_point)
    return successors

In [7]:
point1 = (0,0)
point2 = (3,2)
point3 = (6,6)
assert find_successors(point1, test_world, MOVES) == [(1, 0), (0, 1)]
assert find_successors(point2, test_world, MOVES) == [(3, 1), (4, 2), (3, 3), (2, 2)]
assert find_successors(point3, test_world, MOVES) == [(6, 5), (5, 6)]

<a id="heuristic"></a>
## heuristic

The heuristic i am using is going to be the pythagorean theorom from the 'start' to the 'goal' using their x,y coordinates.

* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** int: the heuristic value for the 'start' node

In [8]:
def heuristic(start: Tuple[int,int], goal: Tuple[int,int]): # you can add formal parameters
    return round(pow(pow(abs(start[0] - goal[0]),2) + pow(abs(start[1] - goal[1]),2), 1/2),0)

In [9]:
test_a = (1,5)
test_b = (10,10)
test_c = (7,2)
test_d = (9,9)
assert heuristic(test_a, test_b) == 10
assert heuristic(test_c, test_b) == 9
assert heuristic(test_d, test_b) == 1

<a id="get_complex_cost_from_path"></a>
## get_complex_cost_from_path

This iterates through the provided path finding the true cost of every point using the cost and heuristic values.

* **path** List[Tuple[int, int]]: the path travelled.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **world** List[List[str]]: the actual context for the navigation problem.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** int: the total cost of the path using the cost and heuristic values.


In [10]:
def get_complex_cost_from_path(path: List[Tuple[int,int]], costs: Dict[str, int], world: List[List[str]], goal: Tuple[int, int]):
    return sum([costs[world[point[0]][point[1]]] + heuristic(point, goal) for point in path])

In [11]:
test_list1 = [(0,0),(1,0),(2,0)]
test_list2 = [(1,0),(2,0),(3,0)]
test_list3 = [(0,0),(0,1),(0,2)]
test_list4 = [(0,0)]
goal = [len(test_world), len(test_world[0])]
assert get_complex_cost_from_path(test_list1, COSTS, test_world, goal) == 31
assert get_complex_cost_from_path(test_list2, COSTS, test_world, goal) == 29
assert get_complex_cost_from_path(test_list3, COSTS, test_world, goal) == 35
assert get_complex_cost_from_path(test_list4, COSTS, test_world, goal) == 11

<a id="get_cost_from_path"></a>
## get_cost_from_path

This iterates through the provided path finding the cost of every point using only the cost values.

* **path** List[Tuple[int, int]]: the path travelled.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **world** List[List[str]]: the actual context for the navigation problem.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** int: the total cost of the path using the cost values.


In [12]:
# this is going to get the total cost of all the points in the passed in list using the passed in world and costs
def get_cost_from_path(path: List[Tuple[int,int]], costs: Dict[str, int], world: List[List[str]], goal: Tuple[int, int]):
    return sum([costs[world[point[0]][point[1]]] for point in path])

In [13]:
test_list1 = [(0,0),(1,0),(2,0)]
test_list2 = [(1,0),(2,0),(3,0)]
test_list3 = [(0,0),(0,1),(0,2)]
goal = [len(test_world), len(test_world[0])]
assert get_cost_from_path(test_list1, COSTS, test_world, goal) == 3
assert get_cost_from_path(test_list2, COSTS, test_world, goal) == 3
assert get_cost_from_path(test_list3, COSTS, test_world, goal) == 7

<a id="get_cheapest_point_index"></a>
## get_cheapest_point_index

This iterates through the provided points and find the cheapest point.

* **points** List[Tuple[int, int]]: the points to have the cost checked.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **world** List[List[str]]: the actual context for the navigation problem.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** int: the index of the cheapest point in the provided list.


In [14]:
def get_cheapest_point_index(points: List[Tuple[int,int]], costs: Dict[str, int], world: List[List[str]], goal: Tuple[int, int]):
    min = 99999
    min_index = -1
    for i in range(len(points)):
        complex_cost = get_complex_cost_from_path([points[i]], costs, world, goal)
        if complex_cost < min:
            min = complex_cost
            min_index = i
    return min_index

In [15]:
test_list1 = [(0,0),(1,0),(2,0)]
test_list2 = [(1,0),(2,0),(3,0)]
test_list3 = [(0,0),(0,1),(0,2)]
goal = [len(test_world), len(test_world[0])]
assert get_cheapest_point_index(test_list1, COSTS, test_world, goal) == 1
assert get_cheapest_point_index(test_list2, COSTS, test_world, goal) == 2
assert get_cheapest_point_index(test_list3, COSTS, test_world, goal) == 0

<a id="get_cleaned_sorted_path"></a>
## get_cleaned_sorted_path

This reworks the provided explored points to be a sorted path of the steps for the A-star solution. (also trims extra points that could exist)

* **explored** List[Tuple[int, int]]: the points that were explored in finding the solution path.
* **start** Tuple[int, int]: the start position for the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.

**returns** List[Tuple[int, int]]: the list of points in order of the a-star solution.


In [16]:
def get_cleaned_sorted_path(path: List[Tuple[int,int]], start: Tuple[int, int], goal: Tuple[int, int], moves: List[Tuple[int, int]]):
    final = [goal]
    start_found = False
    previous_current = [goal]
    current = goal
    working_path = deepcopy(path)
    checked = []
    while (start_found == False and len(working_path) > 0):
        if current == start:
            start_found = True
        moves_accepted = False
        for move in moves:
            new_point = (current[0] + move[0], current[1] + move[1])
            if not new_point in working_path or new_point in final or new_point in checked:
                continue
            moves_accepted = True
            previous_current.append(current)
            current = new_point
            checked.append(current)
            final.append(current)
            working_path.remove(current)
        if moves_accepted == False:
            if current in final:
                final.remove(current)
            current = previous_current.pop()
    final.append(start)
    return final[::-1]

In [17]:
test_list1 = [(0,0),(1,0),(2,0),(3,0)]
test_list2 = [(3,0),(2,0),(1,0)]
test_list3 = [(0,0),(0,1),(0,2),(1,2),(2,2),(3,2),(4,2),(4,1),(4,0)]
assert get_cleaned_sorted_path(test_list1, (0,0), (2,0), MOVES) == [(0, 0), (1, 0), (2, 0)]
assert get_cleaned_sorted_path(test_list2, (1,0), (3,0), MOVES) == [(1, 0), (2, 0), (3, 0)]
assert get_cleaned_sorted_path(test_list3, (0,0), (4,0), MOVES) == [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 1), (4, 0)]

<a id="a_star_search"></a>
## a_star_search

Process to do the search is as follows:
* get the starting point's sucessors
* for each starting point calculate costs and maintain a sorted list of lowest costs
* favoring the lowest cost points, again we find each successor and calculate costs
    * do this until we find the goal
* recurse back through the values we found to find the best path and return that

Variables

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [18]:
def a_star_search(world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    explored = [start]
    frontier = find_successors(start, world, moves)
    goal_found = False
    while len(frontier) > 0 and goal_found == False:
        cheapest_point = frontier.pop(get_cheapest_point_index(frontier, costs, world, goal))
        for s in find_successors(cheapest_point, world, moves):
            if s[0] == goal[0] and s[1] == goal[1]:
                goal_found = True
                explored.append(s)
                break
            if s in explored:
                continue
            frontier.append(s)
        if cheapest_point in explored:
            continue
        explored.append(cheapest_point)
    return get_cleaned_sorted_path(explored, start, goal, moves)

## pretty_print_path

This takes the world, makes a copy of it, then overlays the solution onto it. Logic includes looking at the current point and next point to find what direction of arrow to use.

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **cost** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.


**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [19]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], cost: Dict[str, int]) -> int:
    directions = { (0,1):'⏩', (0,-1):'⏪', (-1,0):'⏫', (1,0):'⏬' }
    finish = '🎁'
    total_cost = get_cost_from_path(path, cost, world, goal)
    p_world = deepcopy(world)
    previous_point = None
    for point in path:
        if point == start:
            previous_point = point
            continue
        diff = (point[0] - previous_point[0], point[1] - previous_point[1])
        p_world[previous_point[0]][previous_point[1]] = directions[diff]
        previous_point = point
        if point == goal:
            p_world[goal[0]][goal[1]] = finish
            break
    # print the pretty path string
    final = ''
    for line in p_world:
        final = final + "".join(line) + "\n"
    print(final)
    return total_cost

## Problems

## Problem 1. 

Execute `a_star_search` and `print_path` for the `test_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [20]:
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(test_world, test_start, test_goal, COSTS, MOVES, heuristic)
test_path_cost = pretty_print_path(test_world, test_path, test_start, test_goal, COSTS)
print(f"total path cost: {test_path_cost}")
print(test_path)

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁

total path cost: 13
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [21]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🗻🗻🗻🗻🗻🗻🗻🌾🌾
⏬🌾🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻⛰⛰⛰🗻🗻⛰⛰
⏬🌾🌾🌾⛰🗻🗻🗻🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🗻🗻⛰🌾
⏬🌾🌾⛰⛰🗻🗻🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🗻🗻🗻⛰🌾
⏬⛰⛰⛰🗻🗻⛰⛰🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🗻⛰🌾🌾
⏬⛰⛰🗻🗻⛰⛰🌾🌾🌾🌾⛰🗻🗻🗻🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
⏩⏬⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰🗻🗻🗻🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
⏬⏪⏩⏩⏩⏩⏬🌾🌾🌾🌾🌾⛰⛰🗻🗻🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
⏬⏩⏫🐊🐊🐊⏩⏬⛰⛰⛰🗻🗻🗻🗻🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
⏩⏫🐊🐊🐊🐊🐊⏬⛰⛰🗻🗻🗻⛰🌾🌾🌾🌾🌾⛰🗻🗻🗻⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊⏬⏪⛰🗻🗻⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🗻🗻⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾⏬⛰⛰🗻🗻⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🗻⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾⏬⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🗻🌾⏩⏬🌲🌲🌲⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🗻⛰🌾🌾🌾
🌾🌾🌾🗻🗻🗻🌲⏩⏩⏩⏬🌲🗻🗻🗻⛰⛰🌾🐊🌾⛰🗻🗻⛰🌾🌾🌾
🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲⏬🌲🌲🌲🗻🗻🗻🌾🌾🗻🗻🗻🌾🌾🌾🌾🌾
🌾🌾🌾🗻🗻🌲🌲🌲🌲🌲⏩⏬🌲🌲🌲🌲🗻🗻🗻🗻🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲⏩⏩⏩⏬🌾🌾⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🌾⏩⏩⏩⏬🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🗻🗻🌲🌲🌲🌲🌲🌾🗻🌾🌾⏩⏬🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🗻🗻🗻🌲🌲🗻🗻🌾🌾🌾🌾⏩⏬🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🗻🗻🗻🗻🌾🌾🌾🌾⛰⛰⏩⏬🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🗻🗻⛰⏩⏬🐊🐊🐊🐊🐊
⛰🗻⛰⛰⛰⛰🌾🌾🌾🌾🌾🗻🗻🗻⛰⛰🗻🗻🌾🗻🗻⏩⏩⏬🐊🐊🐊
⛰🗻🗻🗻⛰🌾🌾🌾🌾🌾⛰⛰🗻🗻🗻🗻⛰⛰⛰⛰🗻🗻🗻⏬🐊🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⏩⏩⏩🎁

total path cost: 123
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (8, 1), (8, 0), (9, 0), (10, 0), (10, 1), (9, 1), (9, 2), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (9, 6), (9, 7), (10, 7), (11, 7), (11, 6), (12, 6),

## Comments

(This is the place to leave me comments)

1. I'm not sure what the emoji was supposed to look like, but the one for the Hill wasn't displaying correctly on my pc so I swapped it to something else.
2. For the full world solution, I couldn't figure out why I get a little loop on the left side, but clearly it shouldn't be there, it should just go right instead of take the like 7 step detour. Any ideas on what I may have coded wrong there?

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.